# Script to get details of the top 250 movies in IMDB from IMDB, Metacritic and Rotten tomatoes

## by Gautam Borgohain


In [1]:
import re
import pandas as pd
import time

pd.options.display.max_columns = None
pd.options.display.max_rows = None

pd.set_option('display.max_colwidth', 1000)

from bs4 import BeautifulSoup
from urllib.parse import quote
import urllib.request as req

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

In [2]:
#
# Function to get the soup objects for a given URL
#
def getSoup(url):
    request= req.Request(url,None,headers)
    try:
        response = req.urlopen(request) 
        time.sleep(4)
        data = response.read()
        soup = BeautifulSoup(data, "html.parser")
        response.close()
    except Exception as e:
        print("There was an error retrieving", url, e)            
    return soup

def cleanText(text):
    return re.sub(r'\n','',text)

url_start = "http://www.imdb.com"

Step 1: Get the Top 250 Movie list from IMDB

In [3]:
imdb250_url = "http://www.imdb.com/chart/top?ref_=nv_mv_250_6"

imdb250_soup = getSoup(imdb250_url)

Step 2: Loop through the 250 movies and extract the movie name, rating and the url to their main page and store the data to a dataframe

In [4]:
table250 = imdb250_soup.find('tbody',{'class': 'lister-list'})
top250_df = pd.DataFrame(columns=['Title','Rating','IMDB_MAIN_URL'])# Initialize an empty dataframe
for movie_row in table250.findAll('tr'):
    title_cell = movie_row.find('td',{'class':'titleColumn'})
    if(title_cell.a):
        movie_url = url_start+ title_cell.a['href']
        movie_name = title_cell.a.text
    rating_cell = movie_row.find('td',{'class':'ratingColumn imdbRating'})
    if(rating_cell):
        rating = cleanText(rating_cell.text)
    top250_df.loc[len(top250_df)] = [movie_name,rating,movie_url]
    
top250_df.head()

,Title,Rating,IMDB_MAIN_URL
0,The Shawshank Redemption,9.2,http://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1
1,The Godfather,9.2,http://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2
2,The Godfather: Part II,9.0,http://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3
3,The Dark Knight,8.9,http://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4
4,Schindler's List,8.9,http://www.imdb.com/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5


Step 3: Loop through the dataframe and go to each movie's main page using the url obtained in the previous step.

From the main page, get Year and Genre information of the movies along with the link to the movie's awards page. Finally add everything into the dataframe

In [7]:
top250_df_copy = top250_df.copy()
# top250_df_copy = top250_df_copy[0:100]

year_list = []
all_genre_list = []
awards_url_list = []
for index, row in top250_df_copy.iterrows():
    temp_url = row['IMDB_MAIN_URL']
    oneMovie = getSoup(temp_url)
    year_soup = oneMovie.find('span',{'id':'titleYear'})
    year = re.sub(r'[()]',"",year_soup.text)
    genre_soup = oneMovie.findAll('span',{'itemprop':'genre'})
    genre_list = []
    for genre in genre_soup:
        genre_list.append(genre.text)
    awards_soup = oneMovie.find(text = 'See more awards')
    if(awards_soup):
        moreawards_url = url_start + awards_soup.parent['href']
    year_list.append(year)
    all_genre_list.append(' '.join(genre_list)) # Using a lit here. Maybe put them in different columns? Dont know if that would be useful
    awards_url_list.append(moreawards_url)
    moreawards_url = " "# Cleaning the variable in case the next movie does not have an awards page
    time.sleep(2)
    if(index%25==0):print("Fetched till ",index)

top250_df_copy.insert(loc = len(top250_df_copy.columns), column = 'YEAR',value = year_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='Genre',value =all_genre_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='IMDB_AWARDS_URL',value = awards_url_list)

top250_df_copy.head()

Fetched till  0
Fetched till  25
Fetched till  50
Fetched till  75
Fetched till  100
Fetched till  125
Fetched till  150
Fetched till  175
Fetched till  200
Fetched till  225


,Title,Rating,IMDB_MAIN_URL,YEAR,Genre,IMDB_AWARDS_URL
0,The Shawshank Redemption,9.2,http://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1,1994,Crime Drama,http://www.imdb.com/title/tt0111161/awards?ref_=tt_awd
1,The Godfather,9.2,http://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2,1972,Crime Drama,http://www.imdb.com/title/tt0068646/awards?ref_=tt_awd
2,The Godfather: Part II,9.0,http://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3,1974,Crime Drama,http://www.imdb.com/title/tt0071562/awards?ref_=tt_awd
3,The Dark Knight,8.9,http://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4,2008,Action Crime Drama,http://www.imdb.com/title/tt0468569/awards?ref_=tt_awd
4,Schindler's List,8.9,http://www.imdb.com/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5,1993,Biography Drama History,http://www.imdb.com/title/tt0108052/awards?ref_=tt_awd


Step 4: Go to each of the awards page in the dataframe and extract the number of awards won by the movie.

The awards that are of interest to use are:
- Oscars - Academy awards
- BAFTA
- SAG
- Golden globes
- Critics Choice

In [ ]:
oscar_nominations_list = []
oscar_wins_list = []
gg_nominations_list = []
gg_wins_list = []
sag_nominations_list = []
sag_wins_list = []
bafta_nominations_list = []
bafta_wins_list = []
cc_wins_list = []
cc_nominations_list = []
total_nominations_list = []
total_wins_list = []

def getCount(td):
    return td.get('rowspan') if (td.get('rowspan')) else 0

def toNumber(numberlist):
    return numberlist[0] if(numberlist) else 0

for index, row in top250_df_copy.iterrows():
    temp_url = row['IMDB_AWARDS_URL']
    if(temp_url != ' '): 
        temp_soup = getSoup(temp_url)
        allstats = temp_soup.find('div',{'class':'desc'})
        total_wins = toNumber(re.findall(r'([0-9]*) wins',allstats.text))
        total_nominations = toNumber(re.findall(r'([0-9]*) nominations',allstats.text))
        td_award = temp_soup.findAll('td', {'class': 'title_award_outcome'})
        oscar_win=0
        oscar_nom=0
        BAFTA_win=0
        BAFTA_nom=0
        gg_win=0
        gg_nom=0
        sag_win=0
        sag_nom=0
        cc_win=0
        cc_nom =0
        for td in td_award:
            if 'Won\nOscar' in td.text:
                oscar_win = getCount(td)
            elif 'Nominated\nOscar' in td.text:
                oscar_nom = getCount(td)
            elif 'Won\nBAFTA' in td.text:
                BAFTA_win = getCount(td)
            elif 'Nominated\nBAFTA' in td.text:
                BAFTA_nom = getCount(td)       
            elif 'Won\nGolden Globe' in td.text:
                gg_win = getCount(td)        
            elif 'Nominated\nGolden Globe' in td.text:
                gg_nom = getCount(td)        
            elif 'Won\nActor' in td.text:
                sag_win = getCount(td)        
            elif 'Nominated\nActor' in td.text:
                sag_nom = getCount(td)        
            elif 'Won\nCritics Choice' in td.text:
                cc_win = getCount(td)    
            elif 'Nominated\nCritics Choice' in td.text:
                cc_nom = getCount(td)
        oscar_wins_list.append(oscar_win)
        oscar_nominations_list.append(oscar_nom)
        cc_wins_list.append(cc_win )
        cc_nominations_list.append(cc_nom )
        bafta_wins_list.append(BAFTA_win)
        bafta_nominations_list.append(BAFTA_nom)
        gg_wins_list.append(gg_win)
        gg_nominations_list.append(gg_nom)
        sag_wins_list.append(sag_win)
        sag_nominations_list.append(sag_nom)
        total_nominations_list.append(total_nominations)
        total_wins_list.append(total_wins)
        if(index%25==0):print("Fetched till ",index)

movies_withoutAwards = top250_df_copy.loc[top250_df_copy.IMDB_AWARDS_URL == ' ']            
top250_df_copy = top250_df_copy.loc[top250_df_copy.IMDB_AWARDS_URL != ' ']# Remove the movies that did not have awards page
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='TOTAL_NOM',value = total_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='TOTAL_WINS',value = total_wins_list)   
top250_df_copy.insert(loc = len(top250_df_copy.columns), column = 'OSCAR_NOM',value = oscar_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='OSCAR_WIN',value =oscar_wins_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='GG_NOM',value = gg_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='GG_WINS',value = gg_wins_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns), column = 'BAFTA_NOM',value = bafta_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='BAFTA_WIN',value =bafta_wins_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='SAG_NOM',value = sag_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='SAG_WINS',value = sag_wins_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='CC_NOM',value = cc_nominations_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='CC_WINS',value = cc_wins_list)
# top250_df_copy = top250_df_copy.append(movies_withoutAwards)# To append the movies without the awards
top250_df_copy.head()


# top250_df_copy['TOTAL_NOM'] = [toNumber(i) for i in top250_df_copy['TOTAL_NOM']]
# top250_df_copy['TOTAL_WINS'] = [toNumber(i) for i in top250_df_copy['TOTAL_WINS']]

Metacritic

Step 5: Go over all the movies in the data frame, go to Metacritic.com and extract the critic and user ratings

In [28]:
metaSearch_Start_url = 'http://www.metacritic.com/search/movie/'
metaSearch_End_url = "/results"
meta_home = "http://www.metacritic.com/"
critic_score_list = []
user_score_list = []
movie_url_list = []
for index, row in top250_df_copy.iterrows():
    critic_score = 0
    user_score = 0
    movie_name = row['Title']
    movie_name_encoded = quote(movie_name.encode('utf8'))
    metaSearch_url = metaSearch_Start_url+movie_name_encoded+metaSearch_End_url
    results_soup = getSoup(metaSearch_url)
    time.sleep(2)
    firstResult = results_soup.find(text = movie_name) # Search the results for the exact movie name
    if(firstResult):
        movie_home_url_part = firstResult.parent['href']
        movie_home_url = meta_home+movie_home_url_part
        movie_soup = getSoup(movie_home_url) # Go to the movie's metacritic homepage
        critic_shell = movie_soup.findAll('a',{'class':'metascore_anchor'}) # This gives a list of all the scores, we need the first two
        if(critic_shell):
            critic_score = cleanText(critic_shell[0].text)
            user_score = cleanText(critic_shell[1].text)
    movie_url_list.append(metaSearch_url)
    critic_score_list.append(critic_score)
    user_score_list.append(user_score)
    time.sleep(2)
    if(index%25==0):print("Fetched till ",index)

top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='MC_MOVIE_URL',value = movie_url_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='MC_CRITIC_RATING',value = critic_score_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='MC_USER_RATING',value = user_score_list)
top250_df_copy.head()

Fetched till  0
Fetched till  25
Fetched till  50
Fetched till  75
Fetched till  100
Fetched till  125
Fetched till  150
Fetched till  175
Fetched till  200
Fetched till  225


,BAFTA_NOM,BAFTA_WIN,CC_NOM,CC_WINS,GG_NOM,GG_WINS,Genre,IMDB_AWARDS_URL,IMDB_MAIN_URL,OSCAR_NOM,OSCAR_WIN,Rating,SAG_NOM,SAG_WINS,TOTAL_NOM,TOTAL_WINS,Title,YEAR,MC_MOVIE_URL,MC_CRITIC_RATING,MC_USER_RATING
0,0,0,0,0,2,0,Crime Drama,http://www.imdb.com/title/tt0111161/awards?ref_=tt_awd,http://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1,7,0,9.2,2,0,28,14,The Shawshank Redemption,1994,http://www.metacritic.com/search/movie/The%20Shawshank%20Redemption/results,80,9.0
1,4,0,0,0,2,5,Crime Drama,http://www.imdb.com/title/tt0068646/awards?ref_=tt_awd,http://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2,8,3,9.2,0,0,27,26,The Godfather,1972,http://www.metacritic.com/search/movie/The%20Godfather/results,100,9.0
2,2,1,0,0,6,0,Crime Drama,http://www.imdb.com/title/tt0071562/awards?ref_=tt_awd,http://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3,5,6,9.0,0,0,20,16,The Godfather: Part II,1974,http://www.metacritic.com/search/movie/The%20Godfather%3A%20Part%20II/results,80,9.0
3,7,1,4,2,0,1,Action Crime Drama,http://www.imdb.com/title/tt0468569/awards?ref_=tt_awd,http://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4,6,2,8.9,0,3,126,144,The Dark Knight,2008,http://www.metacritic.com/search/movie/The%20Dark%20Knight/results,82,8.9
4,6,6,0,0,3,3,Biography Drama History,http://www.imdb.com/title/tt0108052/awards?ref_=tt_awd,http://www.imdb.com/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5,5,7,8.9,0,0,33,76,Schindler's List,1993,http://www.metacritic.com/search/movie/Schindler%27s%20List/results,93,8.5
5,6,2,0,0,5,1,Crime Drama,http://www.imdb.com/title/tt0110912/awards?ref_=tt_awd,http://www.imdb.com/title/tt0110912/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_6,6,1,8.9,3,0,60,54,Pulp Fiction,1994,http://www.metacritic.com/search/movie/Pulp%20Fiction/results,94,9.0
6,1,1,0,0,4,0,Crime Drama,http://www.imdb.com/title/tt0050083/awards?ref_=tt_awd,http://www.imdb.com/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_7,3,0,8.9,0,0,11,16,12 Angry Men,1957,http://www.metacritic.com/search/movie/12%20Angry%20Men/results,0,0
7,6,4,0,4,0,4,Adventure Drama Fantasy,http://www.imdb.com/title/tt0167260/awards?ref_=tt_awd,http://www.imdb.com/title/tt0167260/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_8,0,11,8.9,0,1,106,173,The Lord of the Rings: The Return of the King,2003,http://www.metacritic.com/search/movie/The%20Lord%20of%20the%20Rings%3A%20The%20Return%20of%20the%20King/results,94,8.9
8,0,0,0,0,0,0,Western,http://www.imdb.com/title/tt0060196/awards?ref_=tt_awd,http://www.imdb.com/title/tt0060196/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_9,0,0,8.9,0,0,0,0,"Il buono, il brutto, il cattivo",1966,http://www.metacritic.com/search/movie/Il%20buono%2C%20il%20brutto%2C%20il%20cattivo/results,0,0
9,0,0,0,0,0,0,Drama,http://www.imdb.com/title/tt0137523/awards?ref_=tt_awd,http://www.imdb.com/title/tt0137523/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_10,1,0,8.8,0,0,28,8,Fight Club,1999,http://www.metacritic.com/search/movie/Fight%20Club/results,66,9.0


Rotten Tomatoes

Step 6: Iterate through the list of movies and go to Rottentomatoes.com to get the User and critic ratings.

Problem here is that the web site behaves differently for some movies.For some movies, even sending a request via 'search' url, the user is taken directly to the home pagep of the movie. Thus have to create condition to make sure we are in the right page.

In [30]:
rotten_home = "http://www.rottentomatoes.com"
rotten_search_start = "http://www.rottentomatoes.com/search/?search="
critic_score_list = []
user_score_list = []
movie_url_list = []

def getMovieURL(movie_name,results_soup):
    ul = results_soup.find('ul',{'id': 'movie_results_ul'})
    if(ul):
        lis = ul.findAll('li',{'class':'media bottom_divider clearfix'})
        for li in lis:
            if movie_name.lower() in li.text.lower().encode('ascii','ignore').decode('utf8'):
                return li.find('a')['href']
    return False

for index, row in top250_df_copy.iterrows():
    critic_score = 0
    user_score = 0
    movie_name = row['Title'].encode('ascii','ignore').decode('utf8')
    movie_name_encoded = re.sub(' ','+',movie_name)
    rtSearch_url = rotten_search_start+movie_name_encoded
    results_soup = getSoup(rtSearch_url)
    time.sleep(2)
    if(results_soup):
        resultsdiv = results_soup.findAll('div',{'id':'scoreStats'})
        if(resultsdiv): # If it went to the Home Page directly
#             print("Hom page", movie_name)
            text = resultsdiv[0].text
            critic_rating = toNumber(re.findall(r'Average Rating:  ([^/]+)',text))
            resultsdiv = results_soup.findAll('div',{'class':'audience-info hidden-xs superPageFontColor'})
            text = resultsdiv[0].text
            user_rating = toNumber(re.findall(r'Average Rating: ([^/]+)',text))
            critic_score_list.append(critic_rating)
            user_score_list.append(user_rating)
            movie_url_list.append(rtSearch_url)
        else:
#             print("Went to search page", movie_name,rtSearch_url)
            home_url = getMovieURL(movie_name,results_soup)
            if(home_url):
                complete_home_url = rotten_home+home_url
                results_list= getSoup(complete_home_url)
                resultsdiv = results_list.findAll('div',{'id':'scoreStats'})
                if(len(resultsdiv)>0):
#                     print("Now to Home page", movie_name)
                    text = resultsdiv[0].text
                    critic_rating = toNumber(re.findall(r'Average Rating:  ([^/]+)',text))
                    resultsdiv = results_list.findAll('div',{'class':'audience-info hidden-xs superPageFontColor'})
                    text = resultsdiv[0].text
                    user_rating = toNumber(re.findall(r'Average Rating: ([^/]+)',text))
                    critic_score_list.append(critic_rating)
                    user_score_list.append(user_rating)
                    movie_url_list.append(complete_home_url)
                else:
                    print("Didnt find ratings info", resultsdiv,complete_home_url)
                    critic_score_list.append(0)
                    user_score_list.append(0)
                    movie_url_list.append(complete_home_url)
            else:
                print("Movie not in the result list", home_url,rtSearch_url)
                critic_score_list.append(0)
                user_score_list.append(0)
                movie_url_list.append(rtSearch_url)
                
    time.sleep(2)
    if(index%25==0):print("Fetched till ",index)

top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_MOVIE_URL',value = movie_url_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_CRITIC_RATING',value = critic_score_list)
top250_df_copy.insert(loc = len(top250_df_copy.columns),column ='RT_USER_RATING',value = user_score_list)
top250_df_copy.head()

Fetched till  0
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Il+buono,+il+brutto,+il+cattivo
Didnt find ratings info [] http://www.rottentomatoes.com/m/city_of_god_2011/
Fetched till  25
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Sen+to+Chihiro+no+kamikakushi
Fetched till  50
Didnt find ratings info [] http://www.rottentomatoes.com/m/alienated_2016/
Didnt find ratings info [] http://www.rottentomatoes.com/m/tombstone-for-fireflies/
Movie not in the result list False http://www.rottentomatoes.com/search/?search=WALLE
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Mononoke-hime
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Oldeuboi
Fetched till  75
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Snatch.
Movie not in the result list False http://www.rottentomatoes.com/search/?search=Rashmon
Fetched till  100
Movie not in

,BAFTA_NOM,BAFTA_WIN,CC_NOM,CC_WINS,GG_NOM,GG_WINS,Genre,IMDB_AWARDS_URL,IMDB_MAIN_URL,OSCAR_NOM,OSCAR_WIN,Rating,SAG_NOM,SAG_WINS,TOTAL_NOM,TOTAL_WINS,Title,YEAR,MC_MOVIE_URL,MC_CRITIC_RATING,MC_USER_RATING,RT_MOVIE_URL,RT_CRITIC_RATING,RT_USER_RATING
0,0,0,0,0,2,0,Crime Drama,http://www.imdb.com/title/tt0111161/awards?ref_=tt_awd,http://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1,7,0,9.2,2,0,28,14,The Shawshank Redemption,1994,http://www.metacritic.com/search/movie/The%20Shawshank%20Redemption/results,80,9.0,http://www.rottentomatoes.com/m/shawshank_redemption/,8.2,4.4
1,4,0,0,0,2,5,Crime Drama,http://www.imdb.com/title/tt0068646/awards?ref_=tt_awd,http://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2,8,3,9.2,0,0,27,26,The Godfather,1972,http://www.metacritic.com/search/movie/The%20Godfather/results,100,9.0,http://www.rottentomatoes.com/m/godfather/,9.2,4.4
2,2,1,0,0,6,0,Crime Drama,http://www.imdb.com/title/tt0071562/awards?ref_=tt_awd,http://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3,5,6,9.0,0,0,20,16,The Godfather: Part II,1974,http://www.metacritic.com/search/movie/The%20Godfather%3A%20Part%20II/results,80,9.0,http://www.rottentomatoes.com/search/?search=The+Godfather:+Part+II,9.5,4.3
3,7,1,4,2,0,1,Action Crime Drama,http://www.imdb.com/title/tt0468569/awards?ref_=tt_awd,http://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4,6,2,8.9,0,3,126,144,The Dark Knight,2008,http://www.metacritic.com/search/movie/The%20Dark%20Knight/results,82,8.9,http://www.rottentomatoes.com/m/the_dark_knight/,8.6,4.4
4,6,6,0,0,3,3,Biography Drama History,http://www.imdb.com/title/tt0108052/awards?ref_=tt_awd,http://www.imdb.com/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5,5,7,8.9,0,0,33,76,Schindler's List,1993,http://www.metacritic.com/search/movie/Schindler%27s%20List/results,93,8.5,http://www.rottentomatoes.com/search/?search=Schindler's+List,9,4.3


## Data cleaning and some calculations

In [54]:
top250_df_copy.loc[top250_df_copy.RT_CRITIC_RATING == 'N'] = 0
top250_df_copy.loc[top250_df_copy.RT_USER_RATING == 'N'] = 0

Convert the columns to float datatype

In [56]:
for column in top250_df_copy.columns:
    if column not in ['Genre','IMDB_AWARDS_URL','IMDB_MAIN_URL','Title','RT_MOVIE_URL','MC_MOVIE_URL','YEAR']:
        top250_df_copy[[column]] = top250_df_copy[[column]].astype(float, inplace  = True)

In [97]:
top250_df_copy['AVG_USR_RATING'] = (top250_df_copy.Rating + (2 * top250_df_copy.RT_USER_RATING) + top250_df_copy.MC_USER_RATING) / 3
top250_df_copy['AVG_CRTC_RATING'] = (top250_df_copy.RT_CRITIC_RATING + (top250_df_copy.MC_CRITIC_RATING / 10)) / 2
top250_df_copy['TOTAL_AWARDS'] = (3 * top250_df_copy.TOTAL_WINS + top250_df_copy.TOTAL_NOM) + (3 * (
top250_df_copy.OSCAR_WIN + top250_df_copy.SAG_WINS + top250_df_copy.BAFTA_WIN + top250_df_copy.CC_WINS + top250_df_copy.GG_WINS)) + (
                                 top250_df_copy.BAFTA_NOM + top250_df_copy.GG_NOM + top250_df_copy.CC_NOM + top250_df_copy.OSCAR_NOM + top250_df_copy.SAG_NOM)

top250_df_copy['AVG_RATING'] = (top250_df_copy.AVG_USR_RATING+top250_df_copy.AVG_CRTC_RATING) / 2
top250_df_copy.head()

,Unnamed: 0,BAFTA_NOM,BAFTA_WIN,CC_NOM,CC_WINS,GG_NOM,GG_WINS,Genre,IMDB_AWARDS_URL,IMDB_MAIN_URL,OSCAR_NOM,OSCAR_WIN,Rating,SAG_NOM,SAG_WINS,TOTAL_NOM,TOTAL_WINS,Title,YEAR,MC_MOVIE_URL,MC_CRITIC_RATING,MC_USER_RATING,RT_MOVIE_URL,RT_CRITIC_RATING,RT_USER_RATING,AVG_USR_RATING,AVG_CRTC_RATING,TOTAL_AWARDS,AVG_RATING
0,0,0,0,0,0,2,0,Crime Drama,http://www.imdb.com/title/tt0111161/awards?ref_=tt_awd,http://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1,7,0,9.2,2,0,28,14,The Shawshank Redemption,1994,http://www.metacritic.com/search/movie/The%20Shawshank%20Redemption/results,80,9.0,http://www.rottentomatoes.com/m/shawshank_redemption/,8.2,4.4,9.000000,8.10,81,8.550000
1,1,4,0,0,0,2,5,Crime Drama,http://www.imdb.com/title/tt0068646/awards?ref_=tt_awd,http://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2,8,3,9.2,0,0,27,26,The Godfather,1972,http://www.metacritic.com/search/movie/The%20Godfather/results,100,9.0,http://www.rottentomatoes.com/m/godfather/,9.2,4.4,9.000000,9.60,143,9.300000
2,2,2,1,0,0,6,0,Crime Drama,http://www.imdb.com/title/tt0071562/awards?ref_=tt_awd,http://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3,5,6,9.0,0,0,20,16,The Godfather: Part II,1974,http://www.metacritic.com/search/movie/The%20Godfather%3A%20Part%20II/results,80,9.0,http://www.rottentomatoes.com/search/?search=The+Godfather:+Part+II,9.5,4.3,8.866667,8.75,102,8.808333
3,3,7,1,4,2,0,1,Action Crime Drama,http://www.imdb.com/title/tt0468569/awards?ref_=tt_awd,http://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4,6,2,8.9,0,3,126,144,The Dark Knight,2008,http://www.metacritic.com/search/movie/The%20Dark%20Knight/results,82,8.9,http://www.rottentomatoes.com/m/the_dark_knight/,8.6,4.4,8.866667,8.40,602,8.633333
4,4,6,6,0,0,3,3,Biography Drama History,http://www.imdb.com/title/tt0108052/awards?ref_=tt_awd,http://www.imdb.com/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=2398042102&pf_rd_r=1AXWQX7J7022395YVJHZ&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5,5,7,8.9,0,0,33,76,Schindler's List,1993,http://www.metacritic.com/search/movie/Schindler%27s%20List/results,93,8.5,http://www.rottentomatoes.com/search/?search=Schindler's+List,9.0,4.3,8.666667,9.15,323,8.908333


In [96]:
top250_df_copy = pd.read_csv('/Users/gautamborgohain/Desktop/temp5.csv')
top250_df_copy_backup = top250_df_copy.copy()

In [98]:
top250_df_copy.to_csv('/Users/gautamborgohain/Desktop/temp5.csv')

In [9]:
top250_df = pd.DataFrame(columns=['Title','Rating','IMDB_MAIN_URL'])# Initialize an empty dataframe
list_home_url = 'http://www.imdb.com/search/title?groups=top_1000&sort=user_rating&start=@@@&view=simple'
for i in range(1,1000,100):
    pagesoup = getSoup(re.sub('@@@','1',list_home_url))
    rows = pagesoup.findAll('tr',{'class':re.compile('even|odd')})
    for row in rows[0:10]:
        try:
            movie_name = row.find('a').text
            movie_url = url_start+row.find('a')['href']
            rating = row.find('b').text
            top250_df.loc[len(top250_df)] = [movie_name,rating,movie_url]
        except Exception as e:
            print(row.text, movie_url,e)
top250_df.head()

,Title,Rating,IMDB_MAIN_URL
0,The Shawshank Redemption,9.3,http://www.imdb.com/title/tt0111161/
1,The Godfather,9.2,http://www.imdb.com/title/tt0068646/
2,The Godfather: Part II,9.0,http://www.imdb.com/title/tt0071562/
3,The Dark Knight,9.0,http://www.imdb.com/title/tt0468569/
4,12 Angry Men,8.9,http://www.imdb.com/title/tt0050083/


In [17]:
soup = getSoup('http://www.imdb.com/name/nm0000136/awards?ref_=nm_awd')
soup.find('h3',text = 'Golden Globes, USA').find_next('table').findAll('tr')

<h3>Golden Globes, USA</h3>

In [26]:
rows = soup.find('h3',text = 'Golden Globes, USA').find_next('table').findAll('tr')
rowlis= []
for row in rows:
    if 'Won' in row.text:
        rowlis.append(row.text)

print(rowlis[0])        
'Won' in rowlis[0]



  Best Performance by an Actor in a Motion Picture - Comedy or Musical
Alice in Wonderland (2010)





True

In [12]:
list_home_url = 'http://www.imdb.com/search/title?groups=top_1000&sort=user_rating&start=@@@&view=simple'
re.sub('@@@','1',list_home_url)

'http://www.imdb.com/search/title?groups=top_1000&sort=user_rating&start=1&view=simple'